In [4]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [46]:
#計測地点のosm_idを指定
start_osm = 212430769
mid_osm = [702526911]
end_osm =  702526910
start_mesh = 50325662413
end_mesh = 50325652441

In [47]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [48]:
getdbname = "kddi_06_new_param"
dbname = "kddi_trkn06_monthly_anbun"
trknName = "trkn_06_grp"
sttime = "2022-04-01 06:00:00"

In [49]:
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                cur.execute(sql)
                connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        cur.execute(sql)
        connection.commit()
        cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)


33.7984375000:33.8057291665:132.782031250:132.782031250
50325662214
存在メッシュ：50325662214値21.61544480912339距離0.48427945507792586 km
50325662413
存在メッシュ：50325662413値18.016666666666666距離0.0 km
50325662411
存在メッシュ：50325662411値18.875271191678628距離0.1155404737498997 km
50325662232
存在メッシュ：50325662232値20.807863727722058距離0.37560509101576794 km
50325662233
存在メッシュ：50325662233値19.733875571465497距離0.23108092795718674 km
50325662234
存在メッシュ：50325662234値20.04268690315134距離0.272636972122948 km
50325662214
存在メッシュ：50325662214値21.61544480912339距離0.48427945507792586 km
50325662212
存在メッシュ：50325662212値22.442277124095362距離0.5955444142071099 km
50325652441
存在メッシュ：50325652441値24.4距離0.8589907646091027 km
50325662214
存在メッシュ：50325662214値21.61544480912339距離0.48427945507792586 km
50325652443
存在メッシュ：50325652443値23.599074635463012距離0.7512120453487712 km
50325662212
存在メッシュ：50325662212値22.442277124095362距離0.5955444142071099 km
50325662221
存在メッシュ：50325662221値22.81812568425577距離0.6461215069594988 km
50325652434
存在メッシュ：503256

33.7984375000:33.8057291665:132.782031250:132.782031250
50325662214
存在メッシュ：50325662214値106.49496062503701距離0.48427945507792586 km
50325662413
存在メッシュ：50325662413値99.98333333333333距離0.0 km
50325662411
存在メッシュ：50325662411値101.53689191253252距離0.1155404737498997 km
50325662232
存在メッシュ：50325662232値105.0337238380628距離0.37560509101576794 km
50325662233
存在メッシュ：50325662233値103.0904502289615距離0.23108092795718674 km
50325662234
存在メッシュ：50325662234値103.64921329125468距離0.272636972122948 km
50325662214
存在メッシュ：50325662214値106.49496062503701距離0.48427945507792586 km
50325662212
存在メッシュ：50325662212値107.99103058398109距離0.5955444142071099 km
50325652441
存在メッシュ：50325652441値111.53333333333333距離0.8589907646091027 km
50325662214
存在メッシュ：50325662214値106.49496062503701距離0.48427945507792586 km
50325652443
存在メッシュ：50325652443値110.0841393969779距離0.7512120453487712 km
50325662212
存在メッシュ：50325662212値107.99103058398109距離0.5955444142071099 km
50325662221
50325662221 is kddi none!
50325652434
存在メッシュ：50325652434値109.5055301134

33.7984375000:33.8057291665:132.782031250:132.782031250
50325662214
存在メッシュ：50325662214値104.73636329276958距離0.48427945507792586 km
50325662413
存在メッシュ：50325662413値101.1距離0.0 km
50325662411
存在メッシュ：50325662411値101.96757167409824距離0.1155404737498997 km
50325662232
存在メッシュ：50325662232値103.92034794419956距離0.37560509101576794 km
50325662233
存在メッシュ：50325662233値102.83514320145468距離0.23108092795718674 km
50325662234
存在メッシュ：50325662234値103.14717971676127距離0.272636972122948 km
50325662214
存在メッシュ：50325662214値104.73636329276958距離0.48427945507792586 km
50325662212
存在メッシュ：50325662212値105.5718309321799距離0.5955444142071099 km
50325652441
存在メッシュ：50325652441値107.55距離0.8589907646091027 km
50325662214
存在メッシュ：50325662214値104.73636329276958距離0.48427945507792586 km
50325652443
存在メッシュ：50325652443値106.74070987969762距離0.7512120453487712 km
50325662212
存在メッシュ：50325662212値105.5718309321799距離0.5955444142071099 km
50325662221
存在メッシュ：50325662221値105.95160480367358距離0.6461215069594988 km
50325652434
存在メッシュ：50325652434値10

33.7984375000:33.8057291665:132.782031250:132.782031250
50325662214
存在メッシュ：50325662214値29.623055789463294距離0.48427945507792586 km
50325662413
存在メッシュ：50325662413値25.883333333333333距離0.0 km
50325662411
存在メッシュ：50325662411値26.77556466741887距離0.1155404737498997 km
50325662232
50325662232 is kddi none!
50325662233
存在メッシュ：50325662233値27.66779585059165距離0.23108092795718674 km
50325662234
存在メッシュ：50325662234値27.988701620855267距離0.272636972122948 km
50325662214
存在メッシュ：50325662214値29.623055789463294距離0.48427945507792586 km
50325662212
存在メッシュ：50325662212値30.48227057107908距離0.5955444142071099 km
50325652441
存在メッシュ：50325652441値32.516666666666666距離0.8589907646091027 km
50325662214
存在メッシュ：50325662214値29.623055789463294距離0.48427945507792586 km
50325652443
存在メッシュ：50325652443値31.684373468009433距離0.7512120453487712 km
50325662212
存在メッシュ：50325662212値30.48227057107908距離0.5955444142071099 km
50325662221
存在メッシュ：50325662221値30.87283904873924距離0.6461215069594988 km
50325652434
50325652434 is kddi none!
33.798437

33.7984375000:33.8057291665:132.782031250:132.782031250
50325662214
存在メッシュ：50325662214値101.31115754031701距離0.48427945507792586 km
50325662413
存在メッシュ：50325662413値95.46666666666667距離0.0 km
50325662411
存在メッシュ：50325662411値96.86105834958427距離0.1155404737498997 km
50325662232
存在メッシュ：50325662232値99.99962899558692距離0.37560509101576794 km
50325662233
存在メッシュ：50325662233値98.25544979665328距離0.23108092795718674 km
50325662234
存在メッシュ：50325662234値98.75696584967832距離0.272636972122948 km
50325662214
存在メッシュ：50325662214値101.31115754031701距離0.48427945507792586 km
50325662212
50325662212 is kddi none!
50325652441
存在メッシュ：50325652441値105.83333333333333距離0.8589907646091027 km
50325662214
存在メッシュ：50325662214値101.31115754031701距離0.48427945507792586 km
50325652443
存在メッシュ：50325652443値104.53261381181373距離0.7512120453487712 km
50325662212
50325662212 is kddi none!
50325662221
存在メッシュ：50325662221値103.26433640280354距離0.6461215069594988 km
50325652434
存在メッシュ：50325652434値104.01328484444778距離0.708180008040604 km
33.798437

...

33.7984375000:33.8057291665:132.782031250:132.782031250
50325662214
存在メッシュ：50325662214値75.50308820591776距離0.48427945507792586 km
50325662413
50325662413 is kddi none!
50325662411
50325662411 is kddi none!
50325662232
存在メッシュ：50325662232値73.54633822278348距離0.37560509101576794 km
50325662233
50325662233 is kddi none!
50325662234
存在メッシュ：50325662234値71.69233275750507距離0.272636972122948 km
50325662214
存在メッシュ：50325662214値75.50308820591776距離0.48427945507792586 km
50325662212
存在メッシュ：50325662212値77.50648347561489距離0.5955444142071099 km
50325652441
存在メッシュ：50325652441値82.25距離0.8589907646091027 km
50325662214
存在メッシュ：50325662214値75.50308820591776距離0.48427945507792586 km
50325652443
50325652443 is kddi none!
50325662212
存在メッシュ：50325662212値77.50648347561489距離0.5955444142071099 km
50325662221
存在メッシュ：50325662221値78.41715570493304距離0.6461215069594988 km
50325652434
50325652434 is kddi none!
33.7984375000:33.8057291665:132.782031250:132.782031250
50325662214
存在メッシュ：50325662214値95.27014232299007距離0.4842794

33.7984375000:33.8057291665:132.782031250:132.782031250
50325662214
存在メッシュ：50325662214値99.20697662699118距離0.48427945507792586 km
50325662413
存在メッシュ：50325662413値97.96666666666667距離0.0 km
50325662411
存在メッシュ：50325662411値98.26258258651413距離0.1155404737498997 km
50325662232
存在メッシュ：50325662232値98.92864581042467距離0.37560509101576794 km
50325662233
存在メッシュ：50325662233値98.55849845631013距離0.23108092795718674 km
50325662234
存在メッシュ：50325662234値98.66492951579454距離0.272636972122948 km
50325662214
存在メッシュ：50325662214値99.20697662699118距離0.48427945507792586 km
50325662212
存在メッシュ：50325662212値99.49194233345672距離0.5955444142071099 km
50325652441
存在メッシュ：50325652441値100.16666666666667距離0.8589907646091027 km
50325662214
存在メッシュ：50325662214値99.20697662699118距離0.48427945507792586 km
50325652443
50325652443 is kddi none!
50325662212
存在メッシュ：50325662212値99.49194233345672距離0.5955444142071099 km
50325662221
存在メッシュ：50325662221値99.62147760745457距離0.6461215069594988 km
50325652434
存在メッシュ：50325652434値99.78041843429868距離0.

33.7984375000:33.8057291665:132.782031250:132.782031250
50325662214
存在メッシュ：50325662214値105.65311226041729距離0.48427945507792586 km
50325662413
存在メッシュ：50325662413値102.9距離0.0 km
50325662411
存在メッシュ：50325662411値103.55684367057052距離0.1155404737498997 km
50325662232
存在メッシュ：50325662232値105.03530219031131距離0.37560509101576794 km
50325662233
存在メッシュ：50325662233値104.21368723004193距離0.23108092795718674 km
50325662234
存在メッシュ：50325662234値104.44993193026113距離0.272636972122948 km
50325662214
存在メッシュ：50325662214値105.65311226041729距離0.48427945507792586 km
50325662212
50325662212 is kddi none!
50325652441
存在メッシュ：50325652441値107.78333333333333距離0.8589907646091027 km
50325662214
存在メッシュ：50325662214値105.65311226041729距離0.48427945507792586 km
50325652443
50325652443 is kddi none!
50325662212
50325662212 is kddi none!
50325662221
存在メッシュ：50325662221値106.57317883068826距離0.6461215069594988 km
50325652434
存在メッシュ：50325652434値106.92597930239529距離0.708180008040604 km
33.7984375000:33.8057291665:132.782031250:132.782031

50325662214
存在メッシュ：50325662214値110.65349971082291距離0.48427945507792586 km
50325662413
50325662413 is kddi none!
50325662411
50325662411 is kddi none!
50325662232
存在メッシュ：50325662232値109.68777612000879距離0.37560509101576794 km
50325662233
存在メッシュ：50325662233値108.40347696709625距離0.23108092795718674 km
50325662234
存在メッシュ：50325662234値108.77276049167097距離0.272636972122948 km
50325662214
存在メッシュ：50325662214値110.65349971082291距離0.48427945507792586 km
50325662212
存在メッシュ：50325662212値111.64224435901396距離0.5955444142071099 km
50325652441
存在メッシュ：50325652441値113.98333333333333距離0.8589907646091027 km
50325662214
存在メッシュ：50325662214値110.65349971082291距離0.48427945507792586 km
50325652443
存在メッシュ：50325652443値113.02556879819511距離0.7512120453487712 km
50325662212
存在メッシュ：50325662212値111.64224435901396距離0.5955444142071099 km
50325662221
50325662221 is kddi none!
50325652434
存在メッシュ：50325652434値112.6431690119353距離0.708180008040604 km
33.7984375000:33.8057291665:132.782031250:132.782031250
50325662214
存在メッシュ：503256

33.7984375000:33.8057291665:132.782031250:132.782031250
50325662214
存在メッシュ：50325662214値103.64099116282742距離0.48427945507792586 km
50325662413
存在メッシュ：50325662413値102.1距離0.0 km
50325662411
存在メッシュ：50325662411値102.46765311253775距離0.1155404737498997 km
50325662232
存在メッシュ：50325662232値103.29518620891145距離0.37560509101576794 km
50325662233
存在メッシュ：50325662233値102.83530616289036距離0.23108092795718674 km
50325662234
存在メッシュ：50325662234値102.96753869134069距離0.272636972122948 km
50325662214
存在メッシュ：50325662214値103.64099116282742距離0.48427945507792586 km
50325662212
存在メッシュ：50325662212値103.99503946479975距離0.5955444142071099 km
50325652441
存在メッシュ：50325652441値104.83333333333333距離0.8589907646091027 km
50325662214
存在メッシュ：50325662214値103.64099116282742距離0.48427945507792586 km
50325652443
50325652443 is kddi none!
50325662212
存在メッシュ：50325662212値103.99503946479975距離0.5955444142071099 km
50325662221
存在メッシュ：50325662221値104.15597722946373距離0.6461215069594988 km
50325652434
50325652434 is kddi none!
33.7984375000:33

In [50]:
connection.close()

In [45]:
#CreateGeojsonFile